In [1]:
import torch

PATH="/export/data4/vzhekova/biases-data/Test_De/Statistics"
FASTBPE="/home/vzhekova/fastBPE/fast" # path to the fastBPE tool

In [2]:
%cd $PATH

/export/data4/vzhekova/biases-data/Test_De/Statistics


In [3]:
import csv

In [21]:
# Extract tokens
with open('en.csv') as csv_file, open('tokens.txt', 'w') as out_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        position = int(row[1])
        sentence = row[2].split(" ")
        print(sentence[position], end='\n', file=out_file)

In [11]:
# Extract gender ambiguous words
with open('en.csv') as csv_file, open('words.txt', 'w') as out_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        print(row[3], end='\n', file=out_file)

In [24]:
# Modify gender ambiguous words with gender

# List with source words
words = set()
with open('words.txt', 'r') as fin:
    for line in fin:
        words.add(line.strip())
        
#print(len(words))

with open('en.csv') as csv_file, open('en_disambiguated.txt', 'w') as out_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        modified = row[0] + " " + row[3]
        position = int(row[1])
        sentence = row[2].split(" ")
        sentence[position] = modified
        for token in sentence:
            if (token.replace(',', '') in words): # tokens often contain ","
                token_pos = sentence.index(token)
                sentence[token_pos] = "male " + token
        print(' '.join(sentence), end='\n', file=out_file)

In [17]:
# Modify gender ambiguous words with gender; does not cover all cases unfortunately
with open('en.csv') as csv_file, open('en_disambiguated.txt', 'w') as out_file:
    csv_reader = csv.reader(csv_file)
    
    counter = 0
    temp_sent = []
    temp_gender = []
    temp_words = []
    
    for row in csv_reader:
        counter += 1
        temp_gender.append(row[0])
        if (row[3] == "construction worker"): # special case for 2-part word
            temp_words.append("construction")
        else:
            temp_words.append(row[3])    
        temp_sent.append(row[2].split(" "))
        
        if (counter == 2): # write 2 sentences to file
            for sent in temp_sent:
                for token in sent:
                    if (temp_words[0] in token):
                        sent[sent.index(token)] = temp_gender[0] + " " + temp_words[0]
                    elif (temp_words[1] in token):
                        sent[sent.index(token)] = temp_gender[1] + " " + temp_words[1]
                print(' '.join(sent), end='\n', file=out_file)
            
            # reset all temp variables
            counter = 0
            temp_sent = []
            temp_gender = []
            temp_words = []